# Downsampling and BIDS conversion

## Introduction

In this section we will demonstrate how to prepare the data for further analysis. As the FieldLine data are sampled at 5000 Hz the first step is to downsample to 1000 Hz. Subsequenctly we will convert the data to MEG BIDS (Niso et al., 2018). BIDS (Brain Imaging Data Structure) is a standardised format for organising and describing neuroimaging datasets. Built around straightforward file formats and folder structures, BIDS ensures that the meta data are present and that the file structure is consistent over data sets. 

## Preparation

Import the required modules:

In [ ]:
import json
import os
import array
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import mne
from mne_bids import (
    BIDSPath,
    make_dataset_description,
    print_dir_tree,
    read_raw_bids,
    write_meg_calibration,
    write_meg_crosstalk,
    write_raw_bids,
)
from mne_bids.stats import count_events

### File overview

The chapter relies on the input files:
~~~
<ROOT>/Rec15.fif
~~~ 
and generate the out files:
~~~
<ROOT>/Rec15_fs.fif
<ROOT>/Rec15_event.fif

<ROOT/BIDSRec15/FILE1
<ROOT/BIDSRec15/FILE2
~~~

### Loading the data

The FieldLine data are stored in FIF format which is a binary file structure with embedded lables.

As a first step we will load the data. Make sure the data_data path is set to where the data are stored. We will also define the name of the resampled data with 'rs' added to the filename ( <a>  *_rs.fif  </a>).


In [ ]:
data_path = '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings'
file_name = 'Rec15.fif'
raw_fname = os.path.join(data_path, file_name)

raw = mne.io.read_raw_fif(raw_fname, preload=True);
raw_rest_fname = raw_fname.replace('.fif', f'_rs.fif')

event_fname = raw_fname.replace('.fif', f'_event.fif')

bids_folder = os.path.join(data_path, "BIDSRec15_03")  

Opening raw data file /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15.fif...


/tmp/ipykernel_5812/303311391.py:5: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_fname, preload=True);


Isotrak not found
    Range : 0 ... 1003199 =      0.000 ...   200.640 secs
Ready.
Reading 0 ... 1003199  =      0.000 ...   200.640 secs...


# Lowpass filtering and resampling

The data will now be resampled to 1000 Hz. Prior to resampling data will be lowpass filtered at 1000 Hz/4 = 250 Hz. The downsample is done to avoid alias problems (see Smith 1998, chapter 3). After the lowpass filtering the data are resampled to the desired frequency. 

In [25]:
desired_sfreq = 1000  
current_sfreq = raw.info['sfreq']

lowpass_freq = desired_sfreq / 4.0
raw_rs = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

raw_rs.resample(sfreq = desired_sfreq)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 265 samples (0.053 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


99 events found on stim channel di32
Event IDs: [   1    2    6    7  300  400  500  800  900 1000 1100 1300]


[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.5s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.5s


99 events found on stim channel di32
Event IDs: [   1    2    6    7  300  400  500  800  900 1000 1100 1300]


Measurement date,"September 26, 2024 10:28:45 GMT"
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,"71 Magnetometers, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1000.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


##### As a final step, the resampled data will be saved

In [ ]:
raw_rs.save(raw_rest_fname, overwrite=True)

Overwriting existing file.
Writing /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif
Closing /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif


/tmp/ipykernel_5812/1843415829.py:1: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_rs.save(rawrs_fname, overwrite=True)


[done]


To verify that the data has been resampled write

In [27]:
print(raw_rs.info)

<Info | 15 non-empty values
 bads: []
 ch_names: L108_bz-s1, L208_bz-s65, L115_bz-s2, R504_bz-s66, R508_bz-s3, ...
 chs: 71 Magnetometers, 1 Stimulus
 custom_ref_applied: False
 description: {"chassis":{"version":"0.8.3- ...
 file_id: 4 items (dict)
 gantry_angle: 0.0
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 250.0 Hz
 meas_date: 2024-09-26 10:28:45 UTC
 meas_id: 4 items (dict)
 nchan: 72
 proj_id: 1 item (ndarray)
 proj_name: Emptyroom
 projs: []
 sfreq: 1000.0 Hz
 xplotter_layout: None
>


From now we will be working on the resampled data. Data with the 5000 Hz sample rate can he archived as they now longer will be used. 

## Converting to MEG BIDS format

Read in the resampled data

In [ ]:
del raw, raw_rs
raw = mne.io.read_raw(raw_rest_fname)

Opening raw data file /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif...


/tmp/ipykernel_5812/815680071.py:2: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(rawrs_fname)


Isotrak not found
    Range : 0 ... 200639 =      0.000 ...   200.639 secs
Ready.


### Finding triggers and write a FIF file

The BIDS data will include trigger information. As a first step all the events extract from the trigger channel and also included in an additional FIF file. 


In [ ]:
events = mne.find_events(raw, stim_channel="di32")
mne.write_events(event_fname, events, overwrite=True)

99 events found on stim channel di32
Event IDs: [   1    2    6    7  300  400  500  800  900 1000 1100 1300]
Overwriting existing file.


/tmp/ipykernel_5812/426940137.py:2: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(event_fname,events,overwrite=True)


### Naming the events according to trigger information

The trigger values will now be assigned labels. This will be dependent on specific design of the study and care should be
taken to assing informative labels.

In [ ]:
event_dict = {
    "cue_right": 1,
    "cue_left": 2,
    "trial_start":300,
    "stim_onset":400,
    "catch_onset":500,
    "dot_right": 6,
    "dot_left": 7,
    "resp": 800,
    "blk_start": 900,
    "blk_end":1000,
    "exp_end":1100,
    "cue_noresp":1300
}

### Writing the BIDS file

For the BIDS conversion several parameters needs to be defined according to the subjection and session. Subsequently the BIDS structure can be written.

In [ ]:
raw.info["line_freq"] = 50

subject = '01'  
session = '01'  
task = 'SpAtt'
run = '01'

bids_path = BIDSPath(
    subject=subject, 
    session=session, 
    task=task, 
    run=run, 
    datatype="meg", 
    root=bids_folder
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=event_fname,
    event_id=event_dict,
    #empty_room=raw_er,
    overwrite=True,
)

Opening raw data file /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif...


/tmp/ipykernel_5812/2831929378.py:11: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  write_raw_bids(


Isotrak not found
    Range : 0 ... 200639 =      0.000 ...   200.639 secs
Ready.
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/README'...
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/participants.tsv'...
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/participants.json'...
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json'...
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json'...
Used Annotations descriptions: ['blkend', 'blkstart', 'catchonset', 'cue/left', 'cue/noresp', 'cue/right', 'dot/left', 'dot/right', 'expend', 'resp', 'stimonset', 'trialstart']
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_events.tsv'...
Writing '/home/rakshita/rds_folders/S

/tmp/ipykernel_5812/2831929378.py:11: RuntimeWarning: This filename (/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/Rec15_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  write_raw_bids(


Closing /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_meg.fif
[done]
Writing '/home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/sub-01/ses-01/sub-01_ses-01_scans.tsv'...
Wrote /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03/sub-01/ses-01/sub-01_ses-01_scans.tsv entry with meg/sub-01_ses-01_task-SpAtt_run-1_meg.fif.


BIDSPath(
root: /home/rakshita/rds_folders/Sproject3/EmptyRoom/Recordings/BIDSRec15_03
datatype: meg
basename: sub-01_ses-01_task-SpAtt_run-1_meg.fif)

As last check print the structure of the BIDS file

In [ ]:
print_dir_tree(bids_folder)

|BIDSRec15_03/
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01/
|------ ses-01/
|--------- sub-01_ses-01_scans.tsv
|--------- meg/
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-SpAtt_run-1_channels.tsv
|------------ sub-01_ses-01_task-SpAtt_run-1_events.json
|------------ sub-01_ses-01_task-SpAtt_run-1_events.tsv
|------------ sub-01_ses-01_task-SpAtt_run-1_meg.fif
|------------ sub-01_ses-01_task-SpAtt_run-1_meg.json


## References 
Niso G, Gorgolewski KJ, Bock E, Brooks TL, Flandin G, Gramfort A, Henson RN, Jas M, Litvak V, T Moreau J, Oostenveld R, Schoffelen JM, Tadel F, Wexler J, Baillet S. MEG-BIDS, the brain imaging data structure extended to magnetoencephalography (2018) Sci Data 5:180110. doi: 10.1038/sdata.2018.110.

Smith SW (1998) The Scientist and Engineer's Guide to Digital Signal Processing. California Technical Pub https://www.dspguide.com/pdfbook.htm
